In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
!nvidia-smi

Tue Jun 29 07:15:31 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    25W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Prepare Dataset

In [7]:
from gdrive.MyDrive.utils.cifar10_prepare import cifar10_data_loader
PATH = './content/gdrive/MyDrive/CIFAR10_dataset/data'
batch_size_train,batch_size_test = 64,512
train_loader,test_loader,classes = cifar10_data_loader(PATH,batch_size_train,batch_size_test)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
'''
现在的版本是全精度模型和量化模型的对比
'''
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import seaborn as sns
import math
from torch.utils.data import DataLoader
from torch.autograd import Function

# from torch.utils.tensorboard import SummaryWriter
# make_hist_plot = False

#调整不同超参数，记录不同曲线
train_counters_log = []
train_losses_log = []
test_accuracy_log = []
test_counter_log = []
hps = []
bitGs = [16,16]
log_test = False #是否每隔一定间隔去测试一下？
jy_log_interval = 1024 #每512个样本记录画图

for bitG in bitGs:
  n_epochs = 10
  # bitG = 2
  bitA = 16
  bitW = 8

  hps.append(f'bitG={bitG}')

  hp_log = f'\nn_epochs = {n_epochs}\nbitG = {bitG}\nbitA = {bitA}\nbitW = {bitW}\nbatch_size_train = {batch_size_train}\n'
  print(hp_log)

  learning_rate = 0.01
  momentum = 0.5
  log_interval = 10

  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

  # 设置初始值种子，使得每次开始训练的初始值相同、结果可以复现
  random_seed = 1
  torch.manual_seed(random_seed)
    if device == torch.device('cuda'):
    torch.cuda.manual_seed(random_seed)
  


  
  network = AlexNet(isquant = False).to(device)
  # network = QuantNet()
  # network = JustQuantGrad()
  # optimizer = optim.SGD(network.parameters(), lr=learning_rate,
  #                       momentum=momentum)
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(network.parameters())

  # writer = SummaryWriter('runs/MNIST_tensorboard')
  # images,labels=next(iter(train_loader))
  # grid=torchvision.utils.make_grid(images)
  # writer.add_image('images', grid)
  # writer.add_graph(model=network,input_to_model=images)
  # tb.close()


  train_losses = []
  train_counter = []

  test_accuracy = []
  test_counter = []

  # test_losses = []
  # test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

  def train(epoch):
    network.train()
    for batch_idx, (data, target) in enumerate(train_loader):
      data = data.to(device)
      target = target.to(device)
      optimizer.zero_grad()
      output = network(data)
      # loss = F.nll_loss(output, target)
      loss = criterion(output,target)
      if math.isnan(loss.item()):
        print("get nan loss!")
        network.load_state_dict(torch.load('./model.pth'))
        break
      torch.save(network.state_dict(), './model.pth')
      torch.save(optimizer.state_dict(), './optimizer.pth')
      loss.backward()
      optimizer.step()

      # 相等的样本间隔记录、画图
      if (batch_size_train * batch_idx) % jy_log_interval == 0:
          print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
              epoch, batch_idx * len(data), len(train_loader.dataset),
          100. * batch_idx / len(train_loader), loss.item()))
          train_losses.append(loss.item())
          train_counter.append(
              (batch_size_train * batch_idx) + ((epoch-1)*len(train_loader.dataset)))
          if log_test == True:
              tl,acc = test()
              test_accuracy.append(acc.item())
              test_counter.append(
                  (batch_size_train * batch_idx) + ((epoch-1)*len(train_loader.dataset)))


      # if batch_idx % log_interval == 0:
      #   print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
      #     epoch, batch_idx * len(data), len(train_loader.dataset),
      #     100. * batch_idx / len(train_loader), loss.item()))
      #   train_losses.append(loss.item())
      #   train_counter.append(
      #     (batch_idx*batch_size_train) + ((epoch-1)*len(train_loader.dataset)))



      #   writer.add_scalar('Train/Loss', loss.item(), (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
      #   writer.flush()

  def test():
    network.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
      for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        output = network(data)
        test_loss += criterion(output, target).item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).sum()
    test_loss /= len(test_loader.dataset)
  #   test_losses.append(test_loss)
    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
      test_loss, correct, len(test_loader.dataset),
      100. * correct / len(test_loader.dataset)))
    return test_loss,100. * correct / len(test_loader.dataset)



  for epoch in range(1, n_epochs + 1):
    train(epoch)
    
    # writer.add_scalar('Train/Loss', loss.item(), (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
  #   writer.flush()
    test_loss,accuracy = test()
  # import matplotlib.pyplot as plt

  train_counters_log.append(train_counter)
  train_losses_log.append(train_losses)
  test_counter_log.append(test_counter)
  test_accuracy_log.append(test_accuracy)

  result_log = f'\ntrain_counter = {train_counter}\ntrain_losses = {train_losses}'
  
  if log_test:
    result_log = result_log + f'\ntest_accuracy = {test_accuracy}'

  file = open('run_results',mode='a+')
  file.write(hp_log)
  file.write(result_log)
  file.close()

# end hp explore

fig = plt.figure(dpi=1000)
for i in range(len(bitGs)):
  
  if log_test:
    trl = fig.add_subplot(2,1,1) #训练误差
    trl.plot(train_counters_log[i], train_losses_log[i])
    trl.legend(hps,loc='upper right')
    trl.set_xlabel('number of training examples')
    trl.set_ylabel('cross entropy loss')

    tea = fig.add_subplot(2,1,2) #测试精度
    tea.plot(test_counter_log[i], test_accuracy_log[i])
    tea.legend(hps,loc='upper right')
    tea.set_xlabel('number of training examples')
    tea.set_ylabel('test accuracy')
  else:
    plt.plot(train_counters_log[i], train_losses_log[i])
    plt.legend(hps,loc='upper right')
    plt.xlabel('number of training examples')
    plt.ylabel('cross entropy loss')

plt.savefig('exp')
plt.show()


n_epochs = 3
bitG = 16
bitA = 32
bitW = 32
batch_size_train = 64

Train Epoch: 1 [0/50000 (0%)]	Loss: 2.303882
Train Epoch: 1 [1024/50000 (2%)]	Loss: 2.304816
Train Epoch: 1 [2048/50000 (4%)]	Loss: 2.256161
Train Epoch: 1 [3072/50000 (6%)]	Loss: 2.201043
Train Epoch: 1 [4096/50000 (8%)]	Loss: 2.029397
Train Epoch: 1 [5120/50000 (10%)]	Loss: 2.160360
Train Epoch: 1 [6144/50000 (12%)]	Loss: 2.005521
Train Epoch: 1 [7168/50000 (14%)]	Loss: 1.808043
Train Epoch: 1 [8192/50000 (16%)]	Loss: 1.840828
Train Epoch: 1 [9216/50000 (18%)]	Loss: 2.258432
Train Epoch: 1 [10240/50000 (20%)]	Loss: 1.888587
Train Epoch: 1 [11264/50000 (23%)]	Loss: 1.870700
Train Epoch: 1 [12288/50000 (25%)]	Loss: 1.705283
Train Epoch: 1 [13312/50000 (27%)]	Loss: 1.875639
Train Epoch: 1 [14336/50000 (29%)]	Loss: 1.854397
Train Epoch: 1 [15360/50000 (31%)]	Loss: 1.812885
Train Epoch: 1 [16384/50000 (33%)]	Loss: 1.945669
Train Epoch: 1 [17408/50000 (35%)]	Loss: 1.967223
Train Epoch: 1 [18432/50000 (37%)]	Loss: 1.786184
T

KeyboardInterrupt: ignored